In [2]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
# !pip install nlp
# !pip install matplotlib

In [4]:
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from scipy.sparse import hstack
from sklearn.feature_extraction.text import TfidfVectorizer
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
import csv
from collections import defaultdict
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
# nltk.download('stopwords')
# nltk.download('punkt')
# nltk.download('wordnet')
import matplotlib.pyplot as plt
import nlp
import random
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [5]:
#reading the corresponding csv file
df1 = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')

In [6]:
# train data cleaning

df_train = df1[['reviewText','summary','overall']]
df_train = df_train.dropna()

# Defining a function to clean reviews
def dataCleaning(tweetList):
    
    # remove all special characters
    regex = re.compile('[^a-zA-Z0-9 ]')
    tweetList1 = [regex.sub('', tw) for tw in tweetList]
    
    # change all characters to lowercase
    tweetList1 = [tw.lower() for tw in tweetList1]
        
    # remove stop words using nltk stopwords library
    stop_words = set(stopwords.words('english'))
    for i in range(len(tweetList1)):
        example_sent = tweetList1[i]
        word_tokens = word_tokenize(example_sent)
        filtered_sentence = [w for w in word_tokens if not w in stop_words]
        filtered_sentence = []
        for w in word_tokens:
            if w not in stop_words:
                filtered_sentence.append(w)
        tweetList1[i] = ' '.join(filtered_sentence)
        
    wn = nltk.WordNetLemmatizer()
    for i in range(len(tweetList1)):
        w_list = [wn.lemmatize(word) for word in tweetList1[i].split()]
        tweetList1[i] = " ".join(w_list)
        
    return(tweetList1)

# allRatings = []
# userRatings = defaultdict(list)
# itemRatings = defaultdict(list)

# data = df1

# allRatings = data['overall'].to_list()

# for index_ in data.index:
#     rating = data.loc[index_,'overall']
#     userID = data.loc[index_,'reviewerID']
#     userRatings[userID].append(rating)
    
#     itemID = data.loc[index_,'itemID']
#     itemRatings[itemID].append(rating)

# globalAverage = sum(allRatings)/len(allRatings)

# userAverage = {}
# for u in userRatings:
#     userAverage[u] = sum(userRatings[u]) / len(userRatings[u])
# itemAverage = {}
# for i in itemRatings:
#     itemAverage[i] = sum(itemRatings[i]) / len(itemRatings[i])

# user_avg_array_train = []
# for l in df_train['reviewerID'].tolist():
#     user_avg_array_train.append(userAverage[l])
# item_avg_array_train = []
# for l in df_train['itemID'].tolist():
#     item_avg_array_train.append(itemAverage[l])
     
summary_train = dataCleaning(df_train['summary'].tolist())
df_train['summary'] = summary_train
review_train = dataCleaning(df_train['reviewText'].tolist())
df_train['reviewText'] = review_train

# # One-hot encode category columns
# category_array_train = pd.get_dummies(df_train[['category']], prefix_sep='_', drop_first=True).values

# #Using the Tfidf Vectorizer on the summary and review
# vectorizer = TfidfVectorizer(max_features=2000)
# fit_vectorizer = vectorizer.fit(text_train.tolist())
# tfidf_train = fit_vectorizer.transform(text_train.tolist())

In [7]:
# test data cleaning

df_test = df2[['reviewText','summary']]

for column in df_test.columns:
    df_test[column].fillna(df_test[column].mode()[0], inplace=True)
    
# user_avg_array_test = []
# for l in df_test['reviewerID'].tolist():
#     user_avg_array_test.append(userAverage[l])
# item_avg_array_test = []
# for l in df_test['itemID'].tolist():
#     if l in itemAverage:
#         item_avg_array_test.append(itemAverage[l])
#     else:
#         item_avg_array_test.append(globalAverage)

summary_test = dataCleaning(df_test['summary'].tolist())
df_test['summary'] = summary_test
review_test = dataCleaning(df_test['reviewText'].tolist())
df_test['reviewText'] = review_test

# One-hot encode category columns
# category_array_test = pd.get_dummies(df_test[['category']], prefix_sep='_', drop_first=True).values

text_test = df_test["reviewText"] + ' ' + df_test["summary"]

# tfidf_test = fit_vectorizer.transform(text_test.tolist())

In [8]:
# col1_train = tfidf_train
# col2_train = np.asarray(user_avg_array_train).reshape(-1,1)
# col3_train = np.asarray(item_avg_array_train).reshape(-1,1)
# col4_train = category_array_train
# # clean_matrix_train = hstack((col1_train, col2_train, col3_train, col4_train))
# clean_matrix_train  = col1_train.toarray()

# col1_test = tfidf_test
# col2_test = np.asarray(user_avg_array_test).reshape(-1,1)
# col3_test = np.asarray(item_avg_array_test).reshape(-1,1)
# col4_test = category_array_test
# # clean_matrix_test = hstack((col1_test, col2_test, col3_test, col4_test))
# clean_matrix_test = col1_test.toarray()

# # scaler = StandardScaler()
# # X_train = scaler.fit_transform(clean_matrix_train.todense())
# # X_test = scaler.fit_transform(clean_matrix_test.todense())
# X_train = clean_matrix_train
# X_test = clean_matrix_test
y_train = df1.dropna()['overall'].astype('int64')

In [9]:
text_train = df_train["reviewText"] + ' ' + df_train["summary"]
y_train = df_train['overall'].astype('int64')

X_train, X_val, y_train, y_val = train_test_split(text_train, y_train, test_size=0.3, random_state=1)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=2)

In [10]:
tokenizer = Tokenizer(num_words=10000, oov_token='<UNK>')
tokenizer.fit_on_texts(X_train)

In [11]:
maxlen=50
sequences = tokenizer.texts_to_sequences(X_train)
padded_train = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)

classes = set(y_train)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
train_labels = names_to_ids(y_train)

In [12]:
sequences = tokenizer.texts_to_sequences(X_val)
padded_val = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)

classes = set(y_val)
class_to_index = dict((c,i) for i, c in enumerate(classes))
index_to_class = dict((v,k) for k, v in class_to_index.items())
names_to_ids = lambda labels: np.array([class_to_index.get(x) for x in labels])
val_labels = names_to_ids(y_val)

In [17]:
sequences = tokenizer.texts_to_sequences(X_test)
padded_test = pad_sequences(sequences, truncating = 'post', padding='post', maxlen=maxlen)

In [20]:
model = tf.keras.models.Sequential([
    
tf.keras.layers.Embedding(10000, 16, input_length=maxlen),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20, return_sequences=True)),
tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(20)),
tf.keras.layers.Dense(5, activation='softmax')
    
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
h = model.fit(
     padded_train, train_labels,
     validation_data=(padded_val, val_labels),
     epochs=20,
     callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2)]
)

Epoch 1/20
3281/3281 [==============================] - 64s 17ms/step - loss: 0.8088 - accuracy: 0.6892 - val_loss: 0.7336 - val_accuracy: 0.7100
Epoch 2/20
3281/3281 [==============================] - 53s 16ms/step - loss: 0.6994 - accuracy: 0.7230 - val_loss: 0.7319 - val_accuracy: 0.7099
Epoch 3/20
3281/3281 [==============================] - 54s 16ms/step - loss: 0.6597 - accuracy: 0.7375 - val_loss: 0.7215 - val_accuracy: 0.7138
Epoch 4/20
3281/3281 [==============================] - 55s 17ms/step - loss: 0.6193 - accuracy: 0.7529 - val_loss: 0.7416 - val_accuracy: 0.7062
Epoch 5/20
3281/3281 [==============================] - 55s 17ms/step - loss: 0.5735 - accuracy: 0.7729 - val_loss: 0.7743 - val_accuracy: 0.6998


In [44]:
rnn_probs = model.predict(padded_test)

In [47]:
rnn_predictions = []

for preds in rnn_probs:
    rnn_predictions.append(np.argmax(preds))

In [ ]:
# outfile = open('test_result.csv','w')
# out = csv.writer(outfile)
# out.writerows(map(lambda x: [x], rnn_predictions))
# outfile.close()